In [1]:
%matplotlib inline
import pandas as pd
from pandas import set_option
#Limit dataframe exibition to 8 rows
set_option("display.max_rows", 5)
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gc

In [2]:
arrec_estado = pd.read_csv('./processed/arrec_por_estado.csv', encoding='latin1', sep=',')

In [3]:
arrec_municipio = pd.read_csv('./processed/arrec_por_municipio.csv', encoding='latin1', sep=',')

In [4]:
arrec_estado

,Unnamed: 0,cd_ibge,estado,ano,vl_arrec
0,0,11,Rondônia,2000,117397179
1,1,12,Acre,2000,51507861
...,...,...,...,...,...
457,457,52,Goiás,2016,3773210876
458,458,53,Distrito Federal,2016,14510926130


In [5]:
arrec_municipio.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
arrec_estado.drop('Unnamed: 0', axis=1, inplace=True)

##### Parece que se agrupar os dados de munícipio por estado eu terei o mesmo resultado do dataframe por estado então vou validar isso

In [7]:
validation = pd.DataFrame(arrec_municipio.groupby(by=['estado','ano'])['vl_arrec'].sum()).reset_index()

In [8]:
arrec_estado.drop('cd_ibge', axis=1, inplace=True)

In [9]:
arrec_estado.sort_values(['estado', 'ano'], inplace=True)

In [10]:
arrec_estado['estado'][1]

u' Acre'

Temos um espaço em branco no estado do df arrec_estado

In [11]:
arrec_estado['estado'] = arrec_estado['estado'].str.strip()

In [12]:
len(arrec_estado)

459

In [13]:
len(validation)

459

Ambos dataframes tem o mesmo número de linhas

In [14]:
resultado = pd.merge(arrec_estado, validation, how='inner', left_on=['estado','ano'], right_on=['estado','ano'])

In [15]:
len(resultado)

459

O inner join retornou o mesmo número de linhas então tivemos join em todas as linhas.

In [16]:
resultado.columns

Index([u'estado', u'ano', u'vl_arrec_x', u'vl_arrec_y'], dtype='object')

In [17]:
len(resultado.ix[resultado['vl_arrec_x']==resultado['vl_arrec_y']])

459

##### Como vemos acima, todas as linhas tem o mesmo valor de arrecadação, então não precisamos usar o dataframe  arrec_estado

Vamos liberar memória apagando os dataframes desnecessários

In [18]:
del arrec_estado
del validation
del resultado
gc.collect()

92

In [19]:
emissoes_municipio = pd.read_csv('./processed/emissoes_por_municipio.csv', encoding='latin1', sep=',')

In [20]:
emissoes_municipio.drop(['cd_ibge','Unnamed: 0'], axis=1, inplace=True)

In [21]:
len(emissoes_municipio)

94690

In [22]:
len(arrec_municipio)

94690

##### Vou juntar os valores arrecadados e os valores emitidos 

In [23]:
len(pd.merge(emissoes_municipio, arrec_municipio, how='inner', left_on=['municipio','estado','ano'], right_on=['municipio','estado','ano']))

94690

Como vemos acima o inner join acontece em todas as linhas então podemos seguir em frente com ele

In [24]:
prev = pd.merge(emissoes_municipio, arrec_municipio, how='inner', left_on=['municipio','estado','ano'], right_on=['municipio','estado','ano'])

In [26]:
prev = prev.rename(columns={'vl_arrec_x': 'vl_emissao', 'vl_arrec_y': 'vl_arrecadado'})

Salvando o arquivo para futuras utilizações

In [27]:
prev.to_csv('./processed/prev_com_cd_ibge.csv', encoding='latin1')

In [28]:
prev.drop('cd_ibge', axis=1, inplace=True)

In [29]:
prev.to_csv('./processed/prev.csv', encoding='latin1')